In [37]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset

In [51]:
class BaseModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim, bias=False)

    def forward(self, x):
        return self.linear(x)

In [73]:
def loss_train(model, X, y, num_MC, reg_method, reg_param, criterion):
    """Evaluate model using Monte Carlo averaging"""
    model.train()
    preds = torch.zeros(num_MC, len(X))
    for m in range(num_MC):
        if reg_method == 'Dropout':
            model.add_module('dropout', nn.Dropout(reg_param))
            preds[m] = model(X).squeeze()
        elif reg_method == 'NoiseAddition':
            noise = torch.randn_like(X) * reg_param
            preds[m] = model(X + noise).squeeze()
        else:
            raise ValueError(f"Invalid regularization method: {reg_method}")

    y_pred = preds.mean(dim=0)
    if criterion == 'MSE':
        return nn.MSELoss()(y_pred, y.squeeze())
    elif criterion == 'MAD':
        mean_pred = torch.mean(y_pred)
        return torch.mean(torch.abs(y_pred - mean_pred))
    else:
        raise ValueError(f"Invalid criterion: {criterion}")


def loss_test(model, X, y, num_MC, criterion):
    model.eval()
    with torch.no_grad():
        preds = torch.zeros(num_MC, len(X))
        for m in range(num_MC):
            preds[m] = model(X).squeeze()

        y_pred = preds.mean(dim=0)
        if criterion == 'MSE':
            return nn.MSELoss()(y_pred, y.squeeze())
        elif criterion == 'MAD':
            mean_pred = torch.mean(y_pred)
            return torch.mean(torch.abs(y_pred - mean_pred))

def tune_model(X, y, model, reg_method, num_MC, k_folds, criterion):
    """
    Tune a blackbox regression model using specified regularization method.

    Args:
        X: Input features (n×p matrix)
        y: Target values (n-vector)
        reg_method: Either 'Dropout' or 'NoiseAddition'
        num_MC: Number of Monte Carlo replicates
        k_folds: Number of CV folds
        criterion: Either 'MSE' or 'MAD'
        model_iter: Number of training epochs

    Returns:
        Tuned model with optimal regularization
    """
    if reg_method == 'Dropout':
        param_grid = torch.linspace(0.1, 0.9, 9)
    elif reg_method == 'NoiseAddition':
        param_grid = torch.linspace(0.1, 2.0, 10)
    else:
        raise ValueError(f"Invalid regularization method: {reg_method}")

    dataset = TensorDataset(X, y)
    fold_size = len(dataset) // k_folds
    indices = torch.randperm(len(dataset))

    best_param = None
    best_score = np.inf

    for param in param_grid:
        cv_scores = []

        for k in range(k_folds):
            val_idx = indices[k * fold_size:(k+1) * fold_size]
            train_idx = torch.cat(
                [indices[:k * fold_size], indices[(k + 1) * fold_size:]])

            X_train, y_train = X[train_idx], y[train_idx]
            X_val, y_val = X[val_idx], y[val_idx]

            model = BaseModel(X.shape[1], y.shape[1])
            optimizer = torch.optim.Adam(model.parameters())

            for _ in range(10):
                optimizer.zero_grad()
                loss = loss_train(
                    model, X_train, y_train, num_MC, reg_method, param, criterion)
                loss.backward()
                optimizer.step()

            val_score = loss_test(model, X_val, y_val, num_MC, criterion)
            cv_scores.append(val_score.item())

        mean_score = np.mean(cv_scores)
        if mean_score < best_score:
            best_score = mean_score
            best_param = param

    final_model = BaseModel(X.shape[1], y.shape[1])
    optimizer = torch.optim.Adam(final_model.parameters())

    for _ in range(10):
        optimizer.zero_grad()
        loss = loss_train(final_model, X, y, num_MC, reg_method, best_param, criterion)
        loss.backward()
        optimizer.step()
    print(f'Best parameter: {best_param}')
    return final_model

In [74]:
torch.manual_seed(42)

n_samples, n_features = 100, 10
X = torch.randn(n_samples, n_features)
true_weights = torch.FloatTensor([i + 1 for i in range(n_features)])
y = X @ true_weights + torch.normal(0, 0.1, (n_samples,))
y = y.view(-1, 1)

baseLinearModel = BaseModel(n_features, y.shape[1])
reg_method = 'Dropout'

tuned_model = tune_model(
    X=X,
    y=y,
    model=baseLinearModel,
    reg_method=reg_method,
    num_MC=100,
    k_folds=5,
    criterion='MSE'
)

X_test = torch.randn(10, n_features)
with torch.no_grad():
    y_pred = tuned_model(X_test)


Best parameter: 0.10000000149011612
